<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy

class SimpleModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

def train_local_model(model, data, labels, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    for _ in range(epochs):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

def average_weights(models):
    avg_model = deepcopy(models[0])
    for key in avg_model.state_dict().keys():
        avg_model.state_dict()[key].data.copy_(
            torch.mean(torch.stack([m.state_dict()[key] for m in models]), dim=0)
        )
    return avg_model

# Example usage
global_model = SimpleModel(input_dim=10, output_dim=2)
local_models = [deepcopy(global_model) for _ in range(3)]
data = [torch.randn(64, 10) for _ in range(3)]
labels = [torch.randint(0, 2, (64,)) for _ in range(3)]

for model, d, l in zip(local_models, data, labels):
    train_local_model(model, d, l)

global_model = average_weights(local_models)
print("Federated learning model updated.")